In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
  except RuntimeError as e:
    print(e)

2023-04-28 07:48:52.587395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 07:48:52.687616: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-28 07:48:53.260203: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/bagnol/miniconda3/envs/anagoup/lib/
2023-04-28 07:48:53.260261: W tensorflow/com

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
X_sentence=list()
Y_sentence=list()
count=0
base_path="../dataset"
X_train=list()
Y_train=list()
X_val=list()
Y_val=list()
X_test=list()
Y_test=list()
with open(base_path+"/S1-test.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_test.append(X_sentence)      
                Y_test.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S2-train.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_train.append(X_sentence)      
                Y_train.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
with open(base_path+"/S3-val.txt", encoding="utf8") as file:
    for line in file:
        if(line!='\n'):
            X_sentence.append(line.split(" ")[0])
            Y_sentence.append(line.split(" ")[1][:-1])
        else:
            count+=1
            if(count==2):
                X_val.append(X_sentence)      
                Y_val.append(Y_sentence)        
                X_sentence=list()
                Y_sentence=list()
                count=0
# sentences it is a list of sentences. Every sentence is a list of tuple
tokenized_sentences=X_train+X_test+X_val
tokenized_labels=Y_train+Y_test+Y_val

In [18]:

for el in ["BIOP","APPL","CHAR","CONPRI","ENAT","FEAT","MACEQ","MANP","MANS","MATE","PARA","PRO"]:
    counter=0
    print("occorrenza di "+el)
    for y in Y_train:
        for lab in y:
            if lab!="0" and lab[2:]==el and (lab[0]=="B" or lab[0]=="S"):
                counter+=1
    print("\ttrain: "+str(counter))        
    counter=0
    for y in Y_val:
        for lab in y:
            if lab!="0" and lab[2:]==el and (lab[0]=="B" or lab[0]=="S"):
                counter+=1
    print("\tval: "+str(counter))   
    counter=0
    for y in Y_test:
        for lab in y:
            if lab!="0" and lab[2:]==el and (lab[0]=="B" or lab[0]=="S"):
                counter+=1
    print("\ttest: "+str(counter))   

occorrenza di BIOP
	train: 91
	val: 11
	test: 134
occorrenza di APPL
	train: 1357
	val: 331
	test: 532
occorrenza di CHAR
	train: 1738
	val: 243
	test: 256
occorrenza di CONPRI
	train: 15435
	val: 3239
	test: 2604
occorrenza di ENAT
	train: 860
	val: 197
	test: 181
occorrenza di FEAT
	train: 1357
	val: 487
	test: 782
occorrenza di MACEQ
	train: 1761
	val: 471
	test: 326
occorrenza di MANP
	train: 7417
	val: 1919
	test: 1314
occorrenza di MANS
	train: 38
	val: 50
	test: 14
occorrenza di MATE
	train: 9055
	val: 1795
	test: 2181
occorrenza di PARA
	train: 2576
	val: 481
	test: 422
occorrenza di PRO
	train: 3736
	val: 504
	test: 882


In [3]:
from gensim.models import FastText

model_2=FastText(sentences=X_train, vector_size=100, window=10, min_count=5, sg=1, negative=10, alpha=0.01, sample=0.0001)

In [5]:
model_2.train(X_train, total_examples=len(tokenized_sentences), epochs=30)

defaultdict(int, {})

In [22]:
from modelli.wrapper import Sequence
import tensorflow as tf

lstm=Sequence(embeddings=model_2.wv, initial_vocab=model_2.wv.key_to_index)
lstm.fit_vocab(X_train,Y_train)
lstm.model.call((tf.keras.Input(shape=[None]),tf.keras.Input(shape=[None,None])),training=False)

2023-03-22 10:04:28.369664: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 10:04:28.489483: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-22 10:04:29.069683: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/bagnol/miniconda3/envs/anagoup/lib/
2023-03-22 10:04:29.069750: W tensorflow/com

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<KerasTensor: shape=(None, None, 49) dtype=float32 (created by layer 'crf')>

In [4]:
from modelli.trainer import Trainer

trainer = Trainer(lstm.model, preprocessor=lstm.p)
trainer.train(X_train, Y_train, X_val, Y_val, epochs=20, batch_size=128, verbose=1, shuffle=True)


2023-03-22 10:09:31.914124: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-22 10:09:32.626981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100


Epoch 1/20


2023-03-22 10:09:43.731970: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_23'
2023-03-22 10:09:44.692625: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f6d0f1abbc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-22 10:09:44.692658: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A30, Compute Capability 8.0
2023-03-22 10:09:44.697056

74/74 [==============================] - ETA: 0s - loss: 9.9156 - f1: 74.93


/home/bagnol/miniconda3/envs/anagoup/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        APPL       0.80      0.72      0.76       331
        BIOP       0.00      0.00      0.00        12
        CHAR       0.72      0.64      0.68       243
      CONPRI       0.75      0.75      0.75      3239
        ENAT       0.59      0.31      0.41       197
        FEAT       0.70      0.47      0.56       487
       MACEQ       0.80      0.54      0.65       471
        MANP       0.75      0.83      0.79      1919
        MANS       0.00      0.00      0.00        51
        MATE       0.86      0.85      0.86      1795
        PARA       0.64      0.71      0.67       481
         PRO       0.62      0.78      0.69       504

   micro avg       0.75      0.74      0.75      9730
   macro avg       0.60      0.55      0.57      9730
weighted avg       0.75      0.74      0.74      9730

74/74 [==============================] - 57s 597ms/step - loss: 9.9156 - val_loss: 9.2500 - f1: 0.7493
Epoch 2/20
74/74 [============

In [5]:
lstm.save(model_file="model", preprocessor_file="preprocessor")

In [3]:
from modelli.wrapper import Sequence
import tensorflow as tf

lstm=Sequence(embeddings=None, initial_vocab=None)
lstm.fit_vocab(X_train,Y_train)
lstm.load(model_file="model", preprocessor_file="preprocessor")
lstm.model.call((tf.keras.Input(shape=[None]),tf.keras.Input(shape=[None,None])),training=False)

2023-03-22 10:08:43.617143: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 10:08:44.229328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22124 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:3b:00.0, compute capability: 8.0


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<KerasTensor: shape=(None, None, 49) dtype=float32 (created by layer 'crf')>